In [1]:
'''Retrieve all inputs executed in the current kernel section'''
%history

'''Retrieve all inputs executed in the current kernel section'''
%history


In [2]:
import numpy as np
import pandas as pd

import warnings

In [3]:
print('Pandas version -> %s' % pd.__version__)
print('Numpy version -> %s' % np.__version__)

Pandas version -> 2.1.2
Numpy version -> 1.26.1


In [4]:
df = pd.read_csv('online_retail.csv')

In [5]:
df.head()

,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,Description,Quantity,UnitPrice,Totalsale,CustomerID,Country
0,536365,01-12-2010,08:26:00 AM,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,15.30,17850.0,United Kingdom
1,536365,01-12-2010,08:26:00 AM,71053,WHITE METAL LANTERN,6,3.39,20.34,17850.0,United Kingdom
2,536365,01-12-2010,08:26:00 AM,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,22.00,17850.0,United Kingdom
3,536365,01-12-2010,08:26:00 AM,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,20.34,17850.0,United Kingdom
4,536365,01-12-2010,08:26:00 AM,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,20.34,17850.0,United Kingdom


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   InvoiceDate  541909 non-null  object 
 2   InvoiceTime  541909 non-null  object 
 3   StockCode    541909 non-null  object 
 4   Description  540455 non-null  object 
 5   Quantity     541909 non-null  int64  
 6   UnitPrice    541909 non-null  float64
 7   Totalsale    541909 non-null  float64
 8   CustomerID   406829 non-null  float64
 9   Country      541909 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 41.3+ MB


In [7]:
def assessment(dataframe):
    rows, columns = dataframe.shape
    nulls = dataframe.isnull().any(axis=1).sum()
    duplicates = len(dataframe) - len(dataframe.drop_duplicates())
    neg_qtd = dataframe.query('Quantity < 0').shape[0]
    neg_price = dataframe.query('Totalsale <= 0').shape[0]
    
    return print(f' Number of rows: {rows:,} \n',
                 f'Number of columns: {columns:,} \n',
                 f'Number of rows with nulls values: {nulls:,} \n',
                 f'Number of rows with duplicates: {duplicates:,} \n',
                 f'Number of rows with negative quantity: {neg_qtd:,} \n',
                 f'Number of rows with negative or no sales price: {neg_price:,}')

assessment(df)

 Number of rows: 541,909 
 Number of columns: 10 
 Number of rows with nulls values: 135,080 
 Number of rows with duplicates: 5,268 
 Number of rows with negative quantity: 10,624 
 Number of rows with negative or no sales price: 11,809


In [8]:
#Where are the null values?
df.isnull().sum().to_frame('nulls').query('nulls > 0').rename_axis('column', axis=1).style.format(thousands=',')

column,nulls
Description,"1,454"
CustomerID,"135,080"


In [9]:
#making a copy to avoid changes in the original df
df_retail = df.copy()

In [10]:
#cleaning the dataset
df_retail.drop_duplicates(inplace = True)
df_retail = df_retail.dropna(axis=0,how='any')
df_retail = df_retail.query('Quantity >= 0 & Totalsale > 0')

assessment(df_retail)

 Number of rows: 392,688 
 Number of columns: 10 
 Number of rows with nulls values: 0 
 Number of rows with duplicates: 0 
 Number of rows with negative quantity: 0 
 Number of rows with negative or no sales price: 0


In [11]:
relative_size_of_dropped_data = (df.shape[0] - df_retail.shape[0]) / df.shape[0]

print(f'Relative size of dropped rows after cleanning the dataset: {relative_size_of_dropped_data*100:,.2f}%')

Relative size of dropped rows after cleanning the dataset: 27.54%


In [12]:
#transform data type from object to category if the ratio of values in column over column lenght is small enough
def to_category(df):
    cols = df.select_dtypes(include='object').columns
    for col in cols:
        ratio = len(df[col].value_counts()) / len(df)
        if ratio < 0.1:
            df[col] = df[col].astype('category')
    return df

df_retail = to_category(df_retail)

df_retail.info()

<class 'pandas.core.frame.DataFrame'>
Index: 392688 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   InvoiceNo    392688 non-null  category
 1   InvoiceDate  392688 non-null  category
 2   InvoiceTime  392688 non-null  category
 3   StockCode    392688 non-null  category
 4   Description  392688 non-null  category
 5   Quantity     392688 non-null  int64   
 6   UnitPrice    392688 non-null  float64 
 7   Totalsale    392688 non-null  float64 
 8   CustomerID   392688 non-null  float64 
 9   Country      392688 non-null  category
dtypes: category(6), float64(3), int64(1)
memory usage: 20.1 MB


In [13]:
#adjust type and set format option
df_retail['CustomerID'] = df_retail['CustomerID'].astype(str)
df_retail['CustomerID'] = df_retail['CustomerID'].str.replace('.0','')
pd.options.display.float_format = '{:,.2f}'.format

In [14]:
# Change InvoiceDate type from object to datetime
df_retail['InvoiceDate'] = pd.to_datetime(df_retail['InvoiceDate'], dayfirst=True)

In [15]:
# Retrieve Year-Month from date
df_retail['InvoiceMonth'] = df_retail['InvoiceDate'].dt.to_period('M').astype(str)

In [16]:
# Retrieve part of day from InvoiceTime
warnings.filterwarnings("ignore")

df_retail['Hour'] = pd.to_datetime(df_retail['InvoiceTime']).dt.hour

df_retail['DayPart'] = pd.cut(df_retail['Hour'],
                          bins=[0,5,11,15,20,24],
                          labels=['Night','Morning','Noon','Eve','Night'],
                          ordered=False,
                          include_lowest=True)

warnings.filterwarnings("default")

df_retail.groupby(['Hour','DayPart'],observed=True)['InvoiceNo'].nunique().to_frame().sort_values(
                    ['InvoiceNo'], axis = 0, ascending = False).head(3).style.format(thousands=',')

,,InvoiceNo
Hour,DayPart,
12,Noon,"3,130"
13,Noon,"2,636"
11,Morning,"2,277"


In [17]:
#How the sales vary depends on the time of day
def q25(x):
    return x.quantile(0.25)

def q75(x):
    return x.quantile(0.75)

df_retail.groupby(['DayPart'],observed=True).agg(price_min = ('Totalsale','min'),
                                                 price_q25 = ('Totalsale',q25),
                                                 price_q50 = ('Totalsale','median'),
                                                 price_q75 = ('Totalsale',q75),
                                                 price_max = ('Totalsale','max'),
                                                 price_mean = ('Totalsale','mean'),
                                                 price_std = ('Totalsale','std')
                                                ).reindex(['Morning', 'Noon', 'Eve']).reset_index()

,DayPart,price_min,price_q25,price_q50,price_q75,price_max,price_mean,price_std
0,Morning,0.06,8.25,15.00,20.80,"168,469.60",30.02,547.58
1,Noon,0.08,4.16,10.20,17.70,"38,970.00",19.39,105.53
2,Eve,0.12,3.75,8.85,17.00,"4,176.00",20.05,81.97


In [18]:
#More dataset cleaning
df_retail = df_retail.drop(['Hour','InvoiceTime','Description'],axis=1)

df_retail = df_retail.rename(columns={'Totalsale': 'Revenue'})

df_retail = df_retail[['InvoiceDate','InvoiceMonth','DayPart','InvoiceNo','CustomerID','Country',
                       'StockCode','Quantity','UnitPrice','Revenue']]

df_retail.head(3)

,InvoiceDate,InvoiceMonth,DayPart,InvoiceNo,CustomerID,Country,StockCode,Quantity,UnitPrice,Revenue
0,2010-12-01,2010-12,Morning,536365,17850,United Kingdom,85123A,6,2.55,15.30
1,2010-12-01,2010-12,Morning,536365,17850,United Kingdom,71053,6,3.39,20.34
2,2010-12-01,2010-12,Morning,536365,17850,United Kingdom,84406B,8,2.75,22.00


In [19]:
#Calculate the client's return rate
df_orders_by_user = df_retail.groupby('CustomerID')['InvoiceNo'].nunique().reset_index()
    
all_buyers = df_orders_by_user['CustomerID'].nunique()
one_time_buyers = df_orders_by_user.query('InvoiceNo == 1')['CustomerID'].nunique()
returning_buyers = df_orders_by_user.query('InvoiceNo > 1')['CustomerID'].nunique()

returning_rate = returning_buyers / all_buyers

print(f' Total buyers: {all_buyers:,} \n',
      f'One time buyers: {one_time_buyers:,} \n',
      f'Returning buyers: {returning_buyers:,} \n',
       'Return rate: %2.2f %%'  % (returning_rate*100))

 Total buyers: 4,338 
 One time buyers: 1,493 
 Returning buyers: 2,845 
 Return rate: 65.58 %


In [20]:
#export_data_cleaned
df_retail.to_csv('sales_data_cleaned.csv')